<a href="https://colab.research.google.com/github/yyduyuxuan/Machine-Learning-for-Data-Driven-Inventory-Replenishment-Evidence-from-the-M5-Retail-Dataset/blob/main/Base_Stock_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# µ ( L + R )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# config
L, R = 2, 1
H = L + R
VAL_PARQUET  = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_val_daily_withcroston.parquet"
TEST_PARQUET = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_daily_adapt_withcroston.parquet"

OUT_VAL  = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_val_sum_H.parquet"
OUT_TEST = "/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/ALL_test_sum_H.parquet"

def forward_sum(df, group_cols, value_col, h):
    return (
        df.sort_values(group_cols + ['ds'])
          .groupby(group_cols, group_keys=False)[value_col]
          .apply(lambda s: s.shift(-(h-1)).rolling(h).sum())
    )


def calc_mu_sum_for_df(df: pd.DataFrame, h: int, group_name: str = None) -> pd.DataFrame:
    need = {'unique_id', 'ds', 'model', 'y_hat', 'y_true'}
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"Missing columns: {miss}")

    df = df.copy()
    df['ds'] = pd.to_datetime(df['ds'])

    # H days
    grp = ['unique_id', 'model']
    df['mu_hat_H'] = forward_sum(df, grp, 'y_hat', h)
    df['y_true_H'] = forward_sum(df, grp, 'y_true', h)

    out_cols = ['unique_id', 'ds', 'model', 'mu_hat_H', 'y_true_H']
    out = (df.loc[df['mu_hat_H'].notna(), out_cols]
             .sort_values(['unique_id', 'model', 'ds'])
             .reset_index(drop=True))
    if group_name is not None:
        out.insert(0, 'Group', group_name)
    return out


val_daily_all  = pd.read_parquet(VAL_PARQUET)
test_daily_all = pd.read_parquet(TEST_PARQUET)

# Cal µ(L+R)
val_sum_all  = calc_mu_sum_for_df(val_daily_all,  h=H, group_name='ALL')
test_sum_all = calc_mu_sum_for_df(test_daily_all, h=H, group_name='ALL')

print("[VAL  ALL] rows:", len(val_sum_all),  "| series:", val_sum_all['unique_id'].nunique())
print("[TEST ALL] rows:", len(test_sum_all), "| series:", test_sum_all['unique_id'].nunique())

display(val_sum_all.head())
display(test_sum_all.head())


[VAL  ALL] rows: 55968 | series: 159
[TEST ALL] rows: 229596 | series: 159


,Group,unique_id,ds,model,mu_hat_H,y_true_H
0,ALL,FOODS_1_145_CA_1,2014-10-31,AutoARIMA,4.287336,4.0
1,ALL,FOODS_1_145_CA_1,2014-11-01,AutoARIMA,4.287336,4.0
2,ALL,FOODS_1_145_CA_1,2014-11-02,AutoARIMA,4.287336,5.0
3,ALL,FOODS_1_145_CA_1,2014-11-03,AutoARIMA,4.287336,5.0
4,ALL,FOODS_1_145_CA_1,2014-11-04,AutoARIMA,4.287336,3.0


,Group,unique_id,ds,model,mu_hat_H,y_true_H
0,ALL,FOODS_1_145_CA_1,2015-01-31,AutoARIMA,5.240741,4.0
1,ALL,FOODS_1_145_CA_1,2015-02-01,AutoARIMA,5.240741,3.0
2,ALL,FOODS_1_145_CA_1,2015-02-02,AutoARIMA,5.240741,2.0
3,ALL,FOODS_1_145_CA_1,2015-02-03,AutoARIMA,5.318847,3.0
4,ALL,FOODS_1_145_CA_1,2015-02-04,AutoARIMA,5.396953,2.0


In [ ]:
val_sum_all.to_parquet(OUT_VAL,  index=False)
test_sum_all.to_parquet(OUT_TEST, index=False)

# Val set RMSE

In [ ]:
import numpy as np
import pandas as pd

def baseline_sigma_prior_on_val(
    df: pd.DataFrame,
    group_cols: list | None = None,
    pick_model: str | None = None
) -> pd.DataFrame:

    need = {'y_hat','y_true'}
    if not need.issubset(df.columns):
        raise ValueError(f"df missing: {need - set(df.columns)}")

    if group_cols is None:
        if 'unique_id' in df.columns and 'model' in df.columns:
            group_cols = ['unique_id','model']
        else:
            raise ValueError("Need: ['unique_id','model'].")

    cols_keep = list(set(group_cols + ['y_hat','y_true']))
    df = df[cols_keep].copy()

    def _rmse(g):
        m = np.isfinite(g['y_hat']) & np.isfinite(g['y_true'])
        if m.sum() == 0:
            return pd.Series({'sigma_prior_daily': np.nan, 'n_days': 0})
        e = g.loc[m, 'y_hat'] - g.loc[m, 'y_true']
        return pd.Series({
            'sigma_prior_daily': float(np.sqrt(np.mean(e**2))),
            'n_days': int(m.sum())
        })

    out = (df.groupby(group_cols, sort=False)
             .apply(_rmse)
             .reset_index())

    return out

In [ ]:
prior_all = baseline_sigma_prior_on_val(val_daily_all)
display(prior_all.head(20))


,unique_id,model,sigma_prior_daily,n_days
0,FOODS_1_145_CA_1,AutoARIMA,1.484479,92.0
1,FOODS_1_145_CA_1,SeasonalNaive,1.991831,92.0
2,FOODS_1_145_CA_1,Theta,1.513364,92.0
3,FOODS_1_145_CA_3,AutoARIMA,2.076986,92.0
4,FOODS_1_145_CA_3,SeasonalNaive,3.556989,92.0
5,FOODS_1_145_CA_3,Theta,2.070039,92.0
6,FOODS_1_145_CA_4,AutoARIMA,1.006732,92.0
7,FOODS_1_145_CA_4,CrostonClassic,0.855405,92.0
8,FOODS_1_145_CA_4,SeasonalNaive,0.972446,92.0
9,FOODS_1_145_CA_4,Theta,0.847083,92.0


# Dynamic RMSE (Weekly Update)

In [ ]:
test_daily_all.head()

,unique_id,ds,model,y_hat,y_true,store_tag
0,FOODS_1_145_CA_1,2015-01-29,AutoARIMA,1.746914,5,CA_1
1,FOODS_1_145_CA_1,2015-01-29,SeasonalNaive,2.000000,5,CA_1
2,FOODS_1_145_CA_1,2015-01-29,Theta,1.782293,5,CA_1
3,FOODS_1_145_CA_1,2015-01-30,AutoARIMA,1.746914,4,CA_1
4,FOODS_1_145_CA_1,2015-01-30,SeasonalNaive,0.000000,4,CA_1


In [ ]:
import numpy as np
import pandas as pd

# Dynamic RMSE（baseline, test）
def dynamic_sigma_weekly(
    test_df: pd.DataFrame,
    prior_df: pd.DataFrame,
    window_days: int = 28,      # Near-term window length (days)
    update_every: str = "7D",
    min_obs: int = 7,           # weekly update
    tau: int = 14
) -> pd.DataFrame:

    need_t = {'unique_id','model','ds','y_hat','y_true'}
    need_p = {'unique_id','model','sigma_prior_daily'}
    miss_t = need_t - set(test_df.columns)
    miss_p = need_p - set(prior_df.columns)
    if miss_t:
        raise ValueError(f"test_df missing: {miss_t}")
    if miss_p:
        raise ValueError(f"prior_df missing: {miss_p}")

    # Remove duplicates
    t = test_df[['unique_id','model','ds','y_hat','y_true']].copy()
    t['ds'] = pd.to_datetime(t['ds'])
    t = t.drop_duplicates(subset=['unique_id','model','ds']).sort_values(['unique_id','model','ds'])

    p = prior_df[['unique_id','model','sigma_prior_daily']].drop_duplicates()

    keys = ['unique_id','model']
    t = t.merge(p, on=keys, how='inner')

    out = []
    for (uid, mdl), g in t.groupby(keys, sort=False):
        g = g.sort_values('ds').copy()

        # Daily squared error
        cal = pd.DataFrame({'ds': pd.date_range(g['ds'].min(), g['ds'].max(), freq='D')})
        tmp = cal.merge(g[['ds','y_hat','y_true','sigma_prior_daily']], on='ds', how='left')
        tmp['err2'] = (tmp['y_hat'] - tmp['y_true'])**2

        prior = float(tmp['sigma_prior_daily'].dropna().iloc[0])

        # Update
        updates = pd.date_range(tmp['ds'].min(), tmp['ds'].max(), freq=update_every)
        sig_points = []
        for u in updates:
            start = u - pd.Timedelta(days=window_days)
            win = tmp[(tmp['ds'] >= start) & (tmp['ds'] < u)]['err2'].dropna()
            if len(win) >= min_obs:
                rmse_recent = float(np.sqrt(win.mean()))
                w = len(win) / (len(win) + tau)
                sigma = float(np.sqrt(w * rmse_recent**2 + (1 - w) * prior**2))
            else:
                sigma = prior
            sig_points.append({'ds': u, 'sigma_day': sigma})

        sig = pd.DataFrame(sig_points)
        daily = cal.merge(sig, on='ds', how='left').sort_values('ds')

        daily['sigma_day'] = daily['sigma_day'].ffill().fillna(prior)
        daily['unique_id'] = uid
        daily['model'] = mdl
        out.append(daily[['unique_id','model','ds','sigma_day']])

    sigma_daily = pd.concat(out, ignore_index=True)


    return sigma_daily


sigma_weekly = dynamic_sigma_weekly(
    test_df=test_daily_all,
    prior_df=prior_all,
    window_days=28,
    update_every="7D",
    min_obs=7,
    tau=14
)

# Results
display(sigma_weekly.head())
print(sigma_weekly.shape)


,unique_id,model,ds,sigma_day
0,FOODS_1_145_CA_1,AutoARIMA,2015-01-29,1.484479
1,FOODS_1_145_CA_1,AutoARIMA,2015-01-30,1.484479
2,FOODS_1_145_CA_1,AutoARIMA,2015-01-31,1.484479
3,FOODS_1_145_CA_1,AutoARIMA,2015-02-01,1.484479
4,FOODS_1_145_CA_1,AutoARIMA,2015-02-02,1.484479


(232140, 4)


In [ ]:
sigma_weekly.tail(10)

,unique_id,model,ds,sigma_day
232130,HOUSEHOLD_2_371_WI_3,Theta,2016-01-19,0.752918
232131,HOUSEHOLD_2_371_WI_3,Theta,2016-01-20,0.752918
232132,HOUSEHOLD_2_371_WI_3,Theta,2016-01-21,0.778083
232133,HOUSEHOLD_2_371_WI_3,Theta,2016-01-22,0.778083
232134,HOUSEHOLD_2_371_WI_3,Theta,2016-01-23,0.778083
232135,HOUSEHOLD_2_371_WI_3,Theta,2016-01-24,0.778083
232136,HOUSEHOLD_2_371_WI_3,Theta,2016-01-25,0.778083
232137,HOUSEHOLD_2_371_WI_3,Theta,2016-01-26,0.778083
232138,HOUSEHOLD_2_371_WI_3,Theta,2016-01-27,0.778083
232139,HOUSEHOLD_2_371_WI_3,Theta,2016-01-28,0.707855


In [ ]:
sigma_weekly.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/sigma_weekly.csv")

# SS = zσ√(L + R)

In [ ]:
z = 1.645   # 95%
L, R = 2, 1
H = L + R

# sigma_weekly: ['unique_id','model','ds','sigma_day']
ss_df = sigma_weekly.copy()
ss_df['ds'] = pd.to_datetime(ss_df['ds'])

# SS
ss_df['SS_baseline'] = z * ss_df['sigma_day'] * np.sqrt(H)

ss_df = ss_df[['unique_id','model','ds','SS_baseline']].sort_values(['unique_id','model','ds']).reset_index(drop=True)

display(ss_df.head(10))
print(ss_df.shape)

,unique_id,model,ds,SS_baseline
0,FOODS_1_145_CA_1,AutoARIMA,2015-01-29,4.229612
1,FOODS_1_145_CA_1,AutoARIMA,2015-01-30,4.229612
2,FOODS_1_145_CA_1,AutoARIMA,2015-01-31,4.229612
3,FOODS_1_145_CA_1,AutoARIMA,2015-02-01,4.229612
4,FOODS_1_145_CA_1,AutoARIMA,2015-02-02,4.229612
5,FOODS_1_145_CA_1,AutoARIMA,2015-02-03,4.229612
6,FOODS_1_145_CA_1,AutoARIMA,2015-02-04,4.229612
7,FOODS_1_145_CA_1,AutoARIMA,2015-02-05,4.431742
8,FOODS_1_145_CA_1,AutoARIMA,2015-02-06,4.431742
9,FOODS_1_145_CA_1,AutoARIMA,2015-02-07,4.431742


(232140, 4)


In [ ]:
import numpy as np
import pandas as pd


L, R = 2, 1
H = L + R
z = 1.645   # 95%


S_base = (test_sum_all
          .merge(ss_df, on=['unique_id','model','ds'], how='left'))

if 'sigma_prior_daily' in prior_all.columns:
    S_base = S_base.merge(
        prior_all[['unique_id','model','sigma_prior_daily']],
        on=['unique_id','model'], how='left'
    )
    mask_na = S_base['SS_baseline'].isna() & S_base['sigma_prior_daily'].notna()
    S_base.loc[mask_na, 'SS_baseline'] = z * S_base.loc[mask_na, 'sigma_prior_daily'] * np.sqrt(H)
    S_base.drop(columns=['sigma_prior_daily'], inplace=True)

S_base['SS_baseline'] = S_base['SS_baseline'].fillna(0.0)

# S(t) = μ + SS
S_base['S_baseline'] = S_base['mu_hat_H'].clip(lower=0) + S_base['SS_baseline']

S_base = (S_base[['unique_id','model','ds','mu_hat_H','SS_baseline','S_baseline','y_true_H']]
          .sort_values(['unique_id','model','ds'])
          .reset_index(drop=True))

display(S_base.head(10))
print(S_base.shape)

,unique_id,model,ds,mu_hat_H,SS_baseline,S_baseline,y_true_H
0,FOODS_1_145_CA_1,AutoARIMA,2015-01-31,5.240741,4.229612,9.470353,4.0
1,FOODS_1_145_CA_1,AutoARIMA,2015-02-01,5.240741,4.229612,9.470353,3.0
2,FOODS_1_145_CA_1,AutoARIMA,2015-02-02,5.240741,4.229612,9.470353,2.0
3,FOODS_1_145_CA_1,AutoARIMA,2015-02-03,5.318847,4.229612,9.548459,3.0
4,FOODS_1_145_CA_1,AutoARIMA,2015-02-04,5.396953,4.229612,9.626565,2.0
5,FOODS_1_145_CA_1,AutoARIMA,2015-02-05,5.475058,4.431742,9.906800,4.0
6,FOODS_1_145_CA_1,AutoARIMA,2015-02-06,5.475058,4.431742,9.906800,5.0
7,FOODS_1_145_CA_1,AutoARIMA,2015-02-07,5.475058,4.431742,9.906800,8.0
8,FOODS_1_145_CA_1,AutoARIMA,2015-02-08,5.475058,4.431742,9.906800,7.0
9,FOODS_1_145_CA_1,AutoARIMA,2015-02-09,5.475058,4.431742,9.906800,5.0


(229596, 7)


In [ ]:
# Round up S
S_base['S_baseline'] = np.ceil(S_base['S_baseline']).astype('int64')

print(S_base['S_baseline'].describe())

n_series = S_base['unique_id'].nunique()
print("Distinct series (unique_id):", n_series)

print(
    S_base.groupby('unique_id')['ds'].nunique()
        .sort_values()
        .head()
)

count    229596.000000
mean         26.020854
std          48.268760
min           0.000000
25%           6.000000
50%          11.000000
75%          25.000000
max         567.000000
Name: S_baseline, dtype: float64
Distinct series (unique_id): 159
unique_id
FOODS_1_145_CA_1    361
FOODS_1_145_CA_3    361
FOODS_1_145_CA_4    361
FOODS_1_145_TX_1    361
FOODS_1_145_TX_2    361
Name: ds, dtype: int64


In [ ]:
S_base.tail()

,unique_id,model,ds,mu_hat_H,SS_baseline,S_baseline,y_true_H
229591,HOUSEHOLD_2_371_WI_3,Theta,2016-01-22,2.071476,2.216932,5,0.0
229592,HOUSEHOLD_2_371_WI_3,Theta,2016-01-23,2.071165,2.216932,5,0.0
229593,HOUSEHOLD_2_371_WI_3,Theta,2016-01-24,2.070855,2.216932,5,0.0
229594,HOUSEHOLD_2_371_WI_3,Theta,2016-01-25,2.070544,2.216932,5,0.0
229595,HOUSEHOLD_2_371_WI_3,Theta,2016-01-26,1.710047,2.216932,4,2.0


In [ ]:
S_base.to_csv("/content/drive/MyDrive/Colab Notebooks/Supervised Project/Baseline_Forecast/S_base.csv")